In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pyterrier as pt
if not pt.started():
    pt.java.init()
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', False)


In [2]:
import pyt_splade
splade = pyt_splade.Splade()

dataset_name = 'msmarco-passage'
eval_ds_name = 'dev'
dataset = pt.get_dataset(f'irds:{dataset_name}')
ranker = "splade"
eval_dataset = pt.get_dataset(f'irds:{dataset_name}/{eval_ds_name}')
topics = eval_dataset.get_topics()
qrels = eval_dataset.get_qrels()

work_name="splade"
# root_dir = f'/root/{work_name}'
nfs_data_save = f'/nfs/datasets/cxj/{work_name}'

In [5]:
def create_indexes():
    index_file = f"{nfs_data_save}/{dataset_name}-{ranker}-index"
    if os.path.exists(index_file):
        index = pt.IndexFactory.of(f'{index_file}/data.properties')
    else:
        print(f"indexing into {index_file}")
        pipe = splade.doc_encoder() >> pt.IterDictIndexer(index_file, blocks=True,verbose=True)
        index_ref = pipe.index(dataset.get_corpus_iter(verbose=True), batch_size=16)
        index = pt.IndexFactory.of(index_ref)

    return index

def retrieve():
    index = create_indexes()
    br = splade.query_encoder() >> pt.terrier.Retriever(index, wmodel='Tf') % 10
    pipe = br >> pt.text.get_text(dataset,'text')
    print('start retrieval')
    result = pipe.transform(topics[:10])
    return result

In [6]:
result = retrieve()
print(result)

start retrieval
          qid    docid    docno  rank      score  \
0     1048578  6754205  6754205  0     40.224537   
1     1048578  698926   698926   1     31.593056   
2     1048578  6317754  6317754  2     27.947551   
3     1048578  6347088  6347088  3     27.203906   
4     1048578  6347089  6347089  4     27.203906   
5     1048578  6317752  6317752  5     27.075959   
6     1048578  4608092  4608092  6     26.761775   
7     1048578  8674220  8674220  7     25.964607   
8     1048578  8292876  8292876  8     25.421214   
9     1048578  6754206  6754206  9     25.106816   
1000  1048579  516565   516565   0     13.052883   
1001  1048579  5903631  5903631  1     12.048815   
1002  1048579  6379312  6379312  2     12.048815   
1003  1048579  7453434  7453434  3     12.048815   
1004  1048579  5903625  5903625  4     11.044747   
1005  1048579  516568   516568   5     10.483795   
1006  1048579  2070652  2070652  6     10.040679   
1007  1048579  5903626  5903626  7     10.040679